In [54]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_1samp
from statsmodels.stats.multitest import multipletests

# Metabolomics fold change

In [49]:
df_fc = pd.read_excel('Data/han_nature_metabolomics_peak_fc.xlsx', sheet_name='foldchange.dmrvf.fa.ps', index_col=0)
df_fc.index.name = 'EntryID'
df_fc = df_fc.stack().reset_index()
df_fc.columns = ['EntryID', 'dname', 'FC']
assert df_fc.FC.min()>0
df_fc.head()

/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,EntryID,dname,FC
0,3,m_c18p_0009,0.803225
1,3,m_c18p_0012,3.991658
2,3,m_c18p_0015,10.118235
3,3,m_c18p_0020,0.593802
4,3,m_c18p_0022,0.939388


# Metabolomics metadata

In [50]:
df_info = pd.read_excel('Data/han_nature_metabolomics_peak_fc.xlsx', sheet_name='aggregated_md').rename({'Unnamed: 0':'EntryID'}, axis=1)
df_info.head()

/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,EntryID,experiment,sample_type,media,subculture_time,preculture_time,c18positive,c18negative,hilicpositive,culture_source,...,grouped_taxonomy,kingdom,phylum,class,order,family,genus,species,strain,morphology
0,3,20181030.0,supernatant,pyg,16.00,30.0,s03081,s03182,s02980,c0082,...,Coprococcus,Bacteria,Firmicutes,Clostridia,Clostridiales,Lachnospiraceae,Coprococcus,NaN,HPP0074,NaN
1,4,20190228.0,media_blank,mm,17.25,NaN,s03818,s03658,s03513,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5,20190228.0,media_blank,mm,17.25,NaN,s03819,s03659,s03514,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6,20190228.0,media_blank,mm,17.25,NaN,s03820,s03660,s03515,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,13,20181030.0,supernatant,mm,12.00,30.0,s03029,s03130,s02928,c0206,...,Bacteroides thetaiotaomicron,Bacteria,Bacteroidetes,Bacteroidia,Bacteroidales,Bacteroidaceae,Bacteroides,thetaiotaomicron,NaN,NaN


# Join metabolomics metadata and fold change

In [51]:
df = pd.merge(df_fc, df_info[['EntryID','sample_type','media','taxonomy']], left_on=['EntryID'], right_on=['EntryID'], how='left')
df = df[df.taxonomy.notnull()]
assert len(set(df.sample_type)) == 1 and list(set(df.sample_type))[0]=='supernatant'
df = df.drop(['EntryID','sample_type'], axis=1)
df.head()

,dname,FC,media,taxonomy
0,m_c18p_0009,0.803225,pyg,Coprococcus sp. HPP0074 BEI HM-793
1,m_c18p_0012,3.991658,pyg,Coprococcus sp. HPP0074 BEI HM-793
2,m_c18p_0015,10.118235,pyg,Coprococcus sp. HPP0074 BEI HM-793
3,m_c18p_0020,0.593802,pyg,Coprococcus sp. HPP0074 BEI HM-793
4,m_c18p_0022,0.939388,pyg,Coprococcus sp. HPP0074 BEI HM-793


# Convert metabolite id to BIGG id via inchikey

In [52]:
df_bigg = pd.read_csv('Bigg/bigg_models_metabolites.txt', sep='\t')

# extract InCHI Key
InCHI_Key = []
for dlink in df_bigg.database_links:
    if str(dlink) == 'nan':
        InCHI_Key.append(np.NaN)
    else:
        if 'InChI Key:' in str(dlink):
            InCHI_Key.append(str(str(dlink).split('InChI Key:')[1].split(';')[0].split('/')[-1].strip()))
        else:
            InCHI_Key.append(np.NaN)

df_bigg['inchikey'] = InCHI_Key
df_bigg = df_bigg[df_bigg.inchikey.notnull()]
df_bigg = df_bigg[['universal_bigg_id','inchikey']].drop_duplicates()

print("# unique bigg ids = %d, # unique inchikey = %d."%(len(set(df_bigg.universal_bigg_id)), len(set(df_bigg.inchikey))))

df_id_conv = pd.read_csv('Data/han_nature_metabolites_metadata.tsv', sep='\t')
df_id_conv.inchikey = df_id_conv.inchikey.astype(str)
df_id_conv = pd.merge(df_id_conv, df_bigg, left_on=['inchikey'], right_on=['inchikey'], how='left') # make sure to use left join
df_id_conv.to_csv('Data/han_nature_metabolites_metadata_w_biggids.csv', index=False)

print("%d out of %d metabolites have bigg ids."%(len(set(df_id_conv.dname)), len(set(df_id_conv[df_id_conv.universal_bigg_id.notnull()].dname))))

df_id_conv.head()

# unique bigg ids = 1997, # unique inchikey = 1926.
1511 out of 270 metabolites have bigg ids.


,dname,Compound,Peak,PubChem_CID,mz,rt,adduct,Molecular_formula_as_seen_in_ms,Monoisotopic_mass_as_seen_in_ms,Kingdom,Superclass,Class,Subclass,canonical_smiles,inchikey,mode,QE_rt,QE_ms2,qTOF_ms2,universal_bigg_id
0,m_c18n_0000,O-PHOSPHO-SERINE,NaN,106,184.001648,0.727,[M-H]-,C3H8NO6P,185.008924,Organic compounds,Organic acids and derivatives,Carboxylic acids and derivatives,"Amino acids, peptides, and analogues",C(C(C(=O)O)N)OP(=O)(O)O,BZQFBWGGLXLEPQ-UHFFFAOYSA-N,c18negative,NaN,NaN,NaN,NaN
1,m_c18n_0000,O-PHOSPHO-SERINE,NaN,68841,184.001648,0.727,[M-H]-,C3H8NO6P,185.008924,Organic compounds,Organic acids and derivatives,Carboxylic acids and derivatives,"Amino acids, peptides, and analogues",C(C(C(=O)O)N)OP(=O)(O)O,BZQFBWGGLXLEPQ-REOHCLBHSA-N,c18negative,0.666,*,*,NaN
2,m_c18n_0001,N-ACETYLTRYPTOPHAN,NaN,2002,245.093166,2.981,[M-H]-,C13H14N2O3,246.100442,Organic compounds,Organic acids and derivatives,Carboxylic acids and derivatives,"Amino acids, peptides, and analogues",CC(=O)NC(CC1=CNC2=CC=CC=C21)C(=O)O,DZTHIGRZJZPRDV-UHFFFAOYSA-N,c18negative,3.014,*,*,NaN
3,m_c18n_0001,N-ACETYLTRYPTOPHAN,NaN,439917,245.093166,2.981,[M-H]-,C13H14N2O3,246.100442,Organic compounds,Organic acids and derivatives,Carboxylic acids and derivatives,"Amino acids, peptides, and analogues",CC(=O)NC(CC1=CNC2=CC=CC=C21)C(=O)O,DZTHIGRZJZPRDV-GFCCVEGCSA-N,c18negative,3.053,*,*,NaN
4,m_c18n_0002,5-HYDROXYINDOLE,NaN,16054,132.045488,2.904,[M-H]-,C8H7NO,133.052764,Organic compounds,Organoheterocyclic compounds,Indoles and derivatives,Hydroxyindoles,C1=CC2=C(C=CN2)C=C1O,LMIQERWZRIFWNZ-UHFFFAOYSA-N,c18negative,2.978,*,*,NaN


# Keep only metabolites with Bigg ID

In [53]:
df_id_conv = df_id_conv[df_id_conv.universal_bigg_id.notnull()]
df = pd.merge(df, df_id_conv[['dname','universal_bigg_id']], left_on=['dname'], right_on=['dname'], how='inner')
df['log2FC'] = np.log2(df['FC'])
df = df[['dname','universal_bigg_id','taxonomy','media','FC','log2FC']]

# make sure that taxonomy can be found in strain metadata
df_strain_meta = pd.read_csv('Data/han_nature_strain_metadata.csv')
assert len(set(df.taxonomy) - set(df_strain_meta.taxonomy))==0
df = pd.merge(df, df_strain_meta[['genome_id','taxonomy']], left_on=['taxonomy'], right_on=['taxonomy'], how='left')

df.to_csv("Data/han_nature_log2fc_metabolites_w_biggid.csv", index=False)

df.head()

,dname,universal_bigg_id,taxonomy,media,FC,log2FC,genome_id
0,m_c18p_0022,5aptn,Coprococcus sp. HPP0074 BEI HM-793,pyg,0.939388,-0.090207,G109
1,m_c18p_0022,val__L,Coprococcus sp. HPP0074 BEI HM-793,pyg,0.939388,-0.090207,G109
2,m_c18p_0022,5aptn,Bacteroides thetaiotaomicron VPI 5482,mm,0.939828,-0.089531,G54
3,m_c18p_0022,val__L,Bacteroides thetaiotaomicron VPI 5482,mm,0.939828,-0.089531,G54
4,m_c18p_0022,5aptn,Anaerostipes sp. 3_2_56FAA BEI HM-220 904a,mm,0.485546,-1.042321,G7


# Compute statistics of metabolite fold change

In [57]:
df = pd.read_csv("Data/han_nature_log2fc_metabolites_w_biggid.csv")

res = []
for strain in set(df.genome_id):
    for met in set(df.universal_bigg_id):
        for media in set(df.media):
            df_tmp = df[(df.genome_id==strain) & (df.universal_bigg_id==met) & (df.media==media)]
            if len(df_tmp)>0:
                n_reps = len(df_tmp)
                mean_log2fc = np.mean(df_tmp.log2FC)
                var_log2fc = np.var(df_tmp.log2FC)
                p = ttest_1samp(list(df_tmp.log2FC), 0.0)[1]
                res.append([strain, met, media, n_reps, mean_log2fc, var_log2fc, p])
                
df = pd.DataFrame(res, columns=['genome_id','universal_bigg_id','media','n_reps','log2fc_mean','log2fc_var','log2fc_P'])
df = df[df.log2fc_P.notnull()]
df['log2fc_Padj'] = multipletests(df.log2fc_P, method='fdr_bh')[1]

df.to_csv('Data/han_nature_log2fc_metabolites_w_biggid_stats.csv', index=False)

df.head()

,genome_id,universal_bigg_id,media,n_reps,log2fc_mean,log2fc_var,log2fc_P,log2fc_Padj
0,G124,hom__L,mm,3,0.001837,0.000852,0.937202,0.947879
1,G124,gly,mm,3,-0.390934,0.003383,0.010889,0.026179
2,G124,ins,mm,3,0.258325,0.000304,0.002269,0.008534
3,G124,pala,mm,3,-0.248922,0.001139,0.009067,0.022890
4,G124,pro__L,mm,3,-0.054691,0.001453,0.179617,0.237555
